[View in Colaboratory](https://colab.research.google.com/gist/naviarh/fe0d95b2d197dcf3c1d3df96b64168cc/fastai4.ipynb)

In [0]:
!pip install --upgrade pip
!pip3 install fastai

In [0]:
!pip3 install spacy

### Генератор текста на русском (LSTM)

In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!pwd

/content


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [4]:
import subprocess, os
os.uname()

posix.uname_result(sysname='Linux', nodename='bb733dc4c7ea', release='4.14.33+', version='#1 SMP Wed Jun 20 01:36:48 PDT 2018', machine='x86_64')

In [5]:
from fastai import sgdr
from fastai.io import *
from fastai.nlp import *
from fastai.lm_rnn import *
from fastai.structured import *
from fastai.column_data import *
from fastai.conv_learner import *
from torchtext import vocab, data
!pip3 show fastai torch | grep Name -A 1

Name: fastai
Version: 0.7.0
--
Name: torch
Version: 0.3.1


### Начало

In [6]:
# Организуем файловую структуру
PATH='data/nietzsche/'
TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'
os.makedirs(TRN, exist_ok=True)
os.makedirs(VAL, exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)
%ls {PATH}

1.txt  aphorisms.txt  kastaneda.txt  models/  nietzsche.txt  trn/  val/


In [0]:
# Загрузим текст Активная сторона бецконечности
get_data("http://tululu.org/txt.php?id=79834", f'{PATH}kastaneda.txt')

In [8]:
# Прочитаем
text = open(f'{PATH}kastaneda.txt').read()
text[:57]

'Аннотация: Кроме бесед с Доном Хуаном и магической работы'

In [9]:
# Разделим 80% / 20%
t80 = text[:-len(text)//5]
t20 = text[len(t80):]
len(text), len(t80), len(t20)

(500533, 400426, 100107)

In [10]:
# Создадим тренировочный и валидационный тексты
open(f'{TRN}trn.txt','w').write(t80)
open(f'{VAL}val.txt','w').write(t20)
%ls {TRN} {VAL}

data/nietzsche/trn/:
trn.txt

data/nietzsche/val/:
val.txt


In [11]:
# Предобработчик данных из модуля torchtext
TEXT = data.Field(lower=True, tokenize=list)
TEXT

In [0]:
bs=64 # размер минибатча
bptt=8 # длина рекурсии (размер бэкпропегейшн по времени)
n_fac=42 # размер эмбеддинга
n_hidden=1024 # размер скрытого слоя

In [55]:
# Определим подпапки тренировочной, проверочной, и тестовой выборок
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
FILES

{'test': 'val/', 'train': 'trn/', 'validation': 'val/'}

In [82]:
# Объект данных модели (min_freq=3 - игнор символов, встречающижся реже 3 раз)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)
md

In [83]:
TEXT.vocab.itos[3:10]

['о', 'е', 'а', 'н', 'и', 'т', 'с']

In [84]:
# количество минибатчей
len(md.trn_dl)

778

In [85]:
# Количество уникальных токенов (символов)
md.nt

72

In [86]:
# Количество тренировочных наборов и его размер
len(md.trn_ds), len(md.trn_ds[0].text)

(1, 399090)

### Модель LSTM

In [0]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.3)
        self.l_2 = nn.Linear(n_hidden, 500)
        self.l_out = nn.Linear(500, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(F.sigmoid(self.l_2(outp))), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [91]:
# Модель и оптимизатор
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-3)
m

CharSeqStatefulLSTM(
  (e): Embedding(72, 42)
  (rnn): LSTM(42, 1024, num_layers=2, dropout=0.3)
  (l_2): Linear(in_features=1024, out_features=500, bias=True)
  (l_out): Linear(in_features=500, out_features=72, bias=True)
)

**Обучаем**

In [92]:
fit(m, md, 1, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.570383   2.574516  


[array([2.57452])]

In [0]:
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-4)

In [94]:
fit(m, md, 2, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.153949   2.149751  
 29%|██▉       | 224/778 [00:11<00:28, 19.78it/s, loss=2.1]

    1      2.012429   2.024766  


[array([2.02477])]

In [0]:
lo = LayerOptimizer(optim.Adam, m, 1e-3, 1e-3)

In [97]:
fit(m, md, 2, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.116525   2.122341  
 29%|██▉       | 229/778 [00:11<00:26, 20.60it/s, loss=2.15]

    1      2.155665   2.158213  


[array([2.15821])]

In [0]:
lo = LayerOptimizer(optim.Adam, m, 1e-3, 1e-4)

In [102]:
fit(m, md, 2, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.96673    1.967415  
 30%|███       | 237/778 [00:12<00:27, 19.68it/s, loss=1.94]

    1      1.86706    1.880259  


[array([1.88026])]

In [105]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss   
    0      1.773332   1.80239   
 29%|██▉       | 228/778 [00:11<00:28, 19.49it/s, loss=1.81]

    1      1.743223   1.770381  


    2      1.685698   1.72923   
    3      1.705136   1.739429  
  4%|▍         | 31/778 [00:01<00:37, 19.67it/s, loss=1.7]

    4      1.645671   1.689976  


    5      1.585983   1.653886  
    6      1.555251   1.637951  
  3%|▎         | 22/778 [00:01<00:36, 20.57it/s, loss=1.57]

...

    8      1.573702   1.659033  
    9      1.541857   1.630207  
  4%|▍         | 34/778 [00:01<00:38, 19.30it/s, loss=1.54]

    10     1.518678   1.609519  


    11     1.493876   1.589642  
    12     1.460541   1.576772  
  5%|▌         | 39/778 [00:01<00:35, 21.06it/s, loss=1.46]

    13     1.433445   1.563059  
    14     1.423181   1.557002  


[array([1.557])]

In [0]:
#on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
#cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
#fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

### Результат

In [0]:
# Функция прогнозирования символа
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [107]:
get_next('Карлос Кастане')

'р'

In [0]:
# Функция генерация текста
def get_next_n(inp, n):
    res = inp
    for _ in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [110]:
get_next_n('Карлос Кастане ', 250)

'Карлос Кастане и с грустего дворечкой, которую я смог история. я выстала, что прямо сколько мере; дон хуан просто тему и обычность и говорить, веба, к гронке из них интересное светеюлиз: в том, что-я вдали тебе, как работа в уривает ображдала, прифорбнили дон хуан '